<a href="https://colab.research.google.com/github/dandamudi-geeta/Quantum-Computing/blob/main/2348512_QC(Lab5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install qiskit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 8.5 MB/s eta 0:00:00


In [19]:
!pip install qiskit-aer

In [20]:
from qiskit_aer import AerSimulator

In [22]:
from qiskit_aer import Aer

In [35]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from qiskit.primitives import Sampler


In [24]:
# Define the teleportation circuit
teleport_circuit = QuantumCircuit(3, 2)

In [25]:
# Prepare initial state on Q (qubit 0)
# We'll initialize qubit 0 to an arbitrary state |ψ⟩ = a|0⟩ + b|1⟩
# Here, let's use an example state: 1/√2 (|0⟩ + |1⟩) for demonstration
teleport_circuit.h(0)  # Apply H-gate to Q to create superposition

teleport_circuit.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(3, 'q'), 0), Qubit(QuantumRegister(3, 'q'), 1), Qubit(QuantumRegister(3, 'q'), 2)), clbits=())

In [36]:
# Copy the initial state preparation for comparison later
initial_state_circuit = QuantumCircuit(1)
initial_state_circuit.h(0)  # The same operation as on Q in teleport_circuit

teleport_circuit.barrier()


CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(3, 'q'), 0), Qubit(QuantumRegister(3, 'q'), 1), Qubit(QuantumRegister(3, 'q'), 2)), clbits=())

In [27]:
# Step 2: Entangle A (qubit 1) and B (qubit 2)
teleport_circuit.h(1)      # Apply H-gate to A
teleport_circuit.cx(1, 2)   # Apply CNOT from A to B

teleport_circuit.barrier()


CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(3, 'q'), 0), Qubit(QuantumRegister(3, 'q'), 1), Qubit(QuantumRegister(3, 'q'), 2)), clbits=())

In [28]:
# Step 3: Entangle Q and A
teleport_circuit.cx(0, 1)   # Apply CNOT from Q to A
teleport_circuit.h(0)       # Apply H-gate to Q

teleport_circuit.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(3, 'q'), 0), Qubit(QuantumRegister(3, 'q'), 1), Qubit(QuantumRegister(3, 'q'), 2)), clbits=())

In [29]:
# Step 4: Measure Q and A
teleport_circuit.measure(0, 0)  # Measure Q
teleport_circuit.measure(1, 1)  # Measure A

teleport_circuit.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(3, 'q'), 0), Qubit(QuantumRegister(3, 'q'), 1), Qubit(QuantumRegister(3, 'q'), 2)), clbits=())

In [30]:
# Step 5: Conditional operations on B based on the measurement results
# Conditional X gate on B if the measurement of A is 1
teleport_circuit.x(2).c_if(teleport_circuit.clbits[1], 1)
# Conditional Z gate on B if the measurement of Q is 1
teleport_circuit.z(2).c_if(teleport_circuit.clbits[0], 1)

In [31]:
# Visualize the circuit
print(teleport_circuit)

     ┌───┐ ░                      ░      ┌───┐ ░ ┌─┐    ░            »
q_0: ┤ H ├─░──────────────────────░───■──┤ H ├─░─┤M├────░────────────»
     └───┘ ░ ┌───┐     ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐ ░            »
q_1: ──────░─┤ H ├──■──┤ H ├──■───░─┤ X ├──────░──╫─┤M├─░────────────»
           ░ └───┘┌─┴─┐└───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ░    ┌───┐   »
q_2: ──────░──────┤ X ├─────┤ X ├─░────────────░──╫──╫──░────┤ X ├───»
           ░      └───┘     └───┘ ░            ░  ║  ║  ░    └─╥─┘   »
                                                  ║  ║    ┌────╨────┐»
c: 2/═════════════════════════════════════════════╩══╩════╡ c_1=0x1 ╞»
                                                  0  1    └─────────┘»
«                
«q_0: ───────────
«                
«q_1: ───────────
«        ┌───┐   
«q_2: ───┤ Z ├───
«        └─╥─┘   
«     ┌────╨────┐
«c: 2/╡ c_0=0x1 ╞
«     └─────────┘


In [32]:
# Run the circuit and verify teleportation
simulator = Aer.get_backend('statevector_simulator')

In [38]:
# Execute circuit to get final statevector
job = simulator.run(teleport_circuit)
result = job.result()
final_state = result.get_statevector()



In [39]:
# Get the initial state from the initial_state_circuit for comparison
initial_state = Statevector.from_instruction(initial_state_circuit)


In [40]:
# Compare qubit B's state with the initial state |ψ⟩
teleported_state = Statevector(final_state).trace_out([0, 1])
print("Original State |ψ⟩:", initial_state)
print("Teleported State on B:", teleported_state)
assert initial_state.equiv(teleported_state), "Teleportation failed."

AttributeError: 'Statevector' object has no attribute 'trace_out'